In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
sc = SparkContext("spark://39.99.129.214:7077", "FirstApp")
spark = SparkSession.builder.master("spark://39.99.129.214:7077").config(conf=SparkConf()).getOrCreate()
#spark = SparkSession(sc)

rdd = sc.parallelize([[1, "Delhi, Mumbai, Gandhinagar"],[2, " Delhi, Mandi"], [3, "Hyderbad, Jaipur"]])


In [1]:
documents = rdd.map(lambda l: l[1].replace(" ", "").split(","))

from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(documents)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

NameError: name 'rdd' is not defined

In [3]:
from pyspark.mllib.feature import Normalizer
labels = rdd.map(lambda l: l[0])
features = tfidf

normalizer = Normalizer()
data = labels.zip(normalizer.transform(features))

In [4]:
data

org.apache.spark.api.java.JavaPairRDD@4456ffd1

In [4]:
from pyspark.mllib.linalg.distributed import IndexedRowMatrix
mat = IndexedRowMatrix(data).toBlockMatrix()
dot = mat.multiply(mat.transpose())
dot.toLocalMatrix().toArray()


array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.10794634, 0.        ],
       [0.        , 0.10794634, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])